In [5]:
#!/usr/bin/python3
import sys

sys.path.append("../")
from clustering_imports import *
from clustering_driver import *

k_arr = [3]
R_arr = [30]
C_arr = [35]
d_arr = [2, 4, 8, 16, 32, 64, 128]  # , 256, 512, 1024, 2048, 4096,]# 8172]
N_arr = [
    2,
    4,
    8,
    16,
    32,
    64,
    128,
    256,
    512,
    1024,
    2048,
    4096,
]  # 8172]
base = np.random.randint(0, 2000, (4096, 128, 128))
for k in k_arr:
    for R in R_arr:
        for C in C_arr:
            for N in N_arr:
                for d in d_arr:
                    M = base[:N, :d, :d]
                    node_list, data_list = hierarchify(M, k, R, C)
                    np.random.shuffle(M)
                    tree_match_indices, tree_match_distances = search_tree_associations(
                        node_list, data_list, M
                    )

/bin/bash: line 1: capture: command not found


2023-08-30 00:17:38.045 | INFO     | clustering_driver:hierarchify:256 - 2,4,3,30,0.0003237860000808723,4.620300023816526e-05
2023-08-30 00:17:38.046 | INFO     | clustering_driver:search_tree_associations:75 - 4.2474001020309515e-05
2023-08-30 00:17:38.047 | INFO     | clustering_driver:hierarchify:256 - 2,16,3,30,0.00018640300004335586,2.4781000320217572e-05
2023-08-30 00:17:38.047 | INFO     | clustering_driver:search_tree_associations:75 - 0.00029997700039530173
2023-08-30 00:17:38.048 | INFO     | clustering_driver:hierarchify:256 - 2,64,3,30,0.0002963030001410516,5.186099951970391e-05
2023-08-30 00:17:38.049 | INFO     | clustering_driver:search_tree_associations:75 - 4.0554001316195354e-05
2023-08-30 00:17:38.050 | INFO     | clustering_driver:hierarchify:256 - 2,256,3,30,0.0003267049996793503,2.012999902945012e-05
2023-08-30 00:17:38.051 | INFO     | clustering_driver:search_tree_associations:75 - 0.00010378399929322768
2023-08-30 00:17:38.052 | INFO     | clustering_driver:hie

In [6]:
# if you had a way of capturing log data, this would be the place

pattern = r"(\d+,\d+,\d+,\d+,\d+\.\d+,\d+\.\d+)$"

# Initialize lists to store data
N_values = []
D_values = []
k_values = []
R_values = []
tree_build_times = []
data_build_times = []
search_tree_times = []
# Extract data from log lines
for line in log_lines:
    match = re.search(pattern, line)
    if match:
        metrics = match.group(1)
        values = metrics.split(",")
        N, D, k, R, tree_build, data_build = map(float, values)

        N_values.append(N)
        D_values.append(D)
        k_values.append(k)
        R_values.append(R)
        tree_build_times.append(tree_build)
        data_build_times.append(data_build)
    else:
        search_tree_times.append(float(line.split(" - ")[-1]))

NameError: name 'captured_output' is not defined


---

**Worst Case Scaling**

Worst case scaling is expected to be \(O(N \cdot D \cdot k \cdot R)\), where:

- \(N\) = initial number of input points
- \(D\) = dimensionality of the data
- \(k\) = number of clusters
- \(R\) = number of iterations
- \(C\) = Cutoff
- \(L\) = Levels
- \(M_i\) = input at \(i\)

---

**Analysis**

For each \(i = 0\), while true...

A more detailed analysis would show that at each \(i\), \(M_i\) becomes at most \(k\) clusters \(m\) of size \(\frac{|M_i|}{k}\). The length of the queue is increasing by \(k\) clusters \(m\) for each step \(i\). Assuming linear time kmeans clustering, note that these terms, \(|M_i| = k \cdot |m|\) where \(|m| = \frac{|M_i|}{k}\), are equal.

---

**Complexity Jump in kmedioids**

The only jump in complexity occurs in kmedioids where there is an all-pairs distance matrix computation. For this reason, we chose to only call kmedioids when the input data \(M\) is sufficiently small, below threshold \(C\) which is much less than \(N\). Therefore kmedioids becomes a constant \(O(C^2 \cdot D + C\log(C))\) for the preprocessing, and \(O(C \cdot D \cdot R \cdot k)\) for the actual iterations.

---

**Tree Height Scaling**

There is also a degree of scaling related to the height of the tree. This ends up being something like \(k^{\log(\lfloor N / C \rfloor)}\).

---

